In [2]:
import os
import numpy as np
import pandas as pd
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

base_dir = 'data'
train_dir = os.path.join(base_dir, 'train')
test_dir = os.path.join(base_dir, 'test')

def get_sampled_file_paths(directory, sample_fraction=0.05):
    all_classes = os.listdir(directory)
    sampled_file_paths = []
    labels = []

    for class_name in all_classes:
        class_path = os.path.join(directory, class_name)
        if os.path.isdir(class_path):
            all_files = os.listdir(class_path)
            num_files_to_select = int(len(all_files) * sample_fraction)
            sampled_files = np.random.choice(all_files, size=num_files_to_select, replace=False)
            for file_name in sampled_files:
                sampled_file_paths.append(os.path.join(class_name, file_name))
                labels.append(class_name)

    return sampled_file_paths, labels

train_sampled_file_paths, train_labels = get_sampled_file_paths(train_dir, sample_fraction=0.05)

test_sampled_file_paths, test_labels = get_sampled_file_paths(test_dir, sample_fraction=0.05)

df_train = pd.DataFrame({
    'filename': [os.path.join(train_dir, path) for path in train_sampled_file_paths],
    'class': train_labels
})

df_test = pd.DataFrame({
    'filename': [os.path.join(test_dir, path) for path in test_sampled_file_paths],
    'class': test_labels
})

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

validation_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_dataframe(
    df_train,
    x_col='filename',
    y_col='class',
    target_size=(256, 256),
    batch_size=128,
    class_mode='categorical',
    color_mode='rgba'
)

validation_generator = validation_datagen.flow_from_dataframe(
    df_test,
    x_col='filename',
    y_col='class',
    target_size=(256, 256),
    batch_size=128,
    class_mode='categorical',
    color_mode='rgba'
)

print(train_generator.samples)
print(validation_generator.samples)

Found 2500 validated image filenames belonging to 10 classes.
Found 500 validated image filenames belonging to 10 classes.
2500
500


In [3]:
import tensorflow as tf

model = Sequential([
    tf.keras.Input(shape=(256, 256, 4)), 
    Conv2D(32, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(len(train_generator.class_indices), activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [4]:
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    epochs=10,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // validation_generator.batch_size
)

Epoch 1/10


/Users/dawid/.pyenv/versions/3.11.8/lib/python3.11/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()
2024-08-19 01:06:31.750296: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:450] ShuffleDatasetV3:2: Filling up shuffle buffer (this may take a while): 7 of 8
2024-08-19 01:06:33.288750: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:480] Shuffle buffer filled.


19/19 ━━━━━━━━━━━━━━━━━━━━ 321s 16s/step - accuracy: 0.1202 - loss: 2.3799 - val_accuracy: 0.1745 - val_loss: 2.2179
Epoch 2/10
 1/19 ━━━━━━━━━━━━━━━━━━━━ 4:07 14s/step - accuracy: 0.1484 - loss: 2.2407

2024-08-19 01:11:53.789064: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
/Users/dawid/.pyenv/versions/3.11.8/lib/python3.11/contextlib.py:158: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


19/19 ━━━━━━━━━━━━━━━━━━━━ 16s 129ms/step - accuracy: 0.1484 - loss: 2.2407 - val_accuracy: 0.1293 - val_loss: 2.2258
Epoch 3/10


...

19/19 ━━━━━━━━━━━━━━━━━━━━ 365s 18s/step - accuracy: 0.1923 - loss: 2.1861 - val_accuracy: 0.2266 - val_loss: 2.1591
Epoch 6/10
19/19 ━━━━━━━━━━━━━━━━━━━━ 25s 139ms/step - accuracy: 0.1875 - loss: 2.1664 - val_accuracy: 0.1810 - val_loss: 2.0953
Epoch 7/10


2024-08-19 01:24:25.043851: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:450] ShuffleDatasetV3:2: Filling up shuffle buffer (this may take a while): 5 of 8
2024-08-19 01:24:31.771188: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:480] Shuffle buffer filled.


19/19 ━━━━━━━━━━━━━━━━━━━━ 283s 14s/step - accuracy: 0.1999 - loss: 2.1689 - val_accuracy: 0.2344 - val_loss: 2.0992
Epoch 8/10
19/19 ━━━━━━━━━━━━━━━━━━━━ 14s 132ms/step - accuracy: 0.2109 - loss: 2.1115 - val_accuracy: 0.2414 - val_loss: 2.1460
Epoch 9/10


2024-08-19 01:29:10.413254: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
2024-08-19 01:29:21.256637: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:450] ShuffleDatasetV3:2: Filling up shuffle buffer (this may take a while): 5 of 8
2024-08-19 01:29:26.540569: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:480] Shuffle buffer filled.


19/19 ━━━━━━━━━━━━━━━━━━━━ 291s 14s/step - accuracy: 0.2247 - loss: 2.1134 - val_accuracy: 0.2448 - val_loss: 2.1024
Epoch 10/10
19/19 ━━━━━━━━━━━━━━━━━━━━ 18s 138ms/step - accuracy: 0.2344 - loss: 2.1401 - val_accuracy: 0.2328 - val_loss: 2.0537


In [5]:
test_loss, test_acc = model.evaluate(validation_generator, steps=validation_generator.samples // 32)
print(f'Test accuracy: {test_acc:.2f}')

15/15 ━━━━━━━━━━━━━━━━━━━━ 14s 733ms/step - accuracy: 0.2255 - loss: 2.1012
Test accuracy: 0.23


In [38]:
from tensorflow.keras.preprocessing import image
import numpy as np

def predict_image(img_path, model):
    img = image.load_img(img_path, target_size=(150, 150), color_mode='rgba')  # Obsługa plików PNG
    img_array = image.img_to_array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)
    predictions = model.predict(img_array)
    predicted_class = np.argmax(predictions)
    class_labels = list(train_generator.class_indices.keys())
    return class_labels[predicted_class]

img_path = 'data/train/sztuka nowoczesna/Screenshot 2024-07-25 at 01.22.33.png'
predicted_class = predict_image(img_path, model)
print(f'Ten obrazek przedstawia styl: {predicted_class}')

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step
Ten obrazek przedstawia styl: renesans


In [ ]:
from ipywidgets import widgets
from IPython.display import display
import PIL.Image

def on_upload_change(change):
    img = PIL.Image.open(change['new'][0])
    img_path = 'uploaded_image.png'
    img.save(img_path)
    display(img)
    predicted_class = predict_image(img_path, model)
    print(f'Ten obrazek przedstawia styl: {predicted_class}')

upload_widget = widgets.FileUpload(accept='image/*', multiple=False)
upload_widget.observe(on_upload_change, names='value')
display(upload_widget)